<a href="https://colab.research.google.com/github/koki0702/zerobook3/blob/master/notebook/ja/step08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 前ステップまでに実装したコード

In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

***

# ステップ8 再帰からループへ

前ステップで私たちは、`Variable`クラスに`backward`メソッドを追加しました。ここでは処理効率の改善と今後の拡張を見据えて、`backward`メソッドを別の実装方式へと変更します。

## 8.1 現時点のVariableクラス

再掲になりますが、私たちは`Variable`クラスの`backward`メソッドを次のように実装しました。

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

ここで注目したいのは、`backward`メソッドの中で、（入力側へ）1つ前の変数の`backward`メソッドが呼ばれている点です。これによって、「`backward`メソッドの中で`backward`メソッドが呼ばれ、その呼ばれた先の`backward`メソッドでまた`backward`メソッドが呼ばれ、...」という処理が続きます（関数`self.creator`が`None`になる変数が見つかるまで続きます）。これは再帰的な構造です。


## 8.2 ループを使った実装

ここでは、上の「再帰を使った実装」を「ループを使った実装」に書き換えます。そのコードを示すと、次のようになります。

In [3]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 関数を取得
            x, y = f.input, f.output  # 関数の入出力を取得
            x.grad = f.backward(y.grad)  # backwardメソッドを呼ぶ

            if x.creator is not None:
                funcs.append(x.creator)  # 1つ前の関数をリストに追加

これがループを使った実装です。重要な点は、`funcs`というリストに処理すべき関数を順に追加していくことです。`while`ループの中では、`funcs.pop()`によって処理すべき関数が`f`として取り出され、その関数`f`の`backward`メソッドが呼ばれます。このとき、`f.input`と`f.output`によって、関数`f`の入出力の変数を取得することで、`f.backward()`の引数と戻り値が正しく設定されます。

<br><div class="alert alert-info">
<b>【ノート】</b>リストの<code>pop</code>メソッドは、リストの末尾が削除され、その要素が取得されます。たとえば、<code>funcs = [1, 2, 3]</code>のとき<code>x = funcs.pop()</code>とすれば、<code>3</code>が取り出され、<code>funcs</code>は<code>[1, 2]</code>となります。
</div><br>



In [3]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # ★出力変数に生みの親を覚えさせる
        self.input = input
        self.output = output      # ★出力も覚える
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

順伝播の計算により`output`という`Variable`インスタンスが生成されます。このとき、その生成された`output`に対して、「私（関数である自分自身）が生みの親であること」を覚えさせます。これこそが、動的に「つながり」を作る仕掛けの核心部です。なお、ここでは次のステップを見据えて、インスタンス変数の`output`に出力を設定しています。

<br><div class="alert alert-info">
<b>【ノート】</b>DeZeroの動的計算グラフの仕組みは、実際の計算が行われるときに、変数という「箱」にその「つながり」を記録することによって行われます。同様のアプローチは、ChainerやPyTorchでも行われています。
</div><br>

このように、「つながり」を持った`Variable`と`Function`があれば、計算グラフを逆向きに辿ることができます。具体的なコードで示すと、次のようになります。

In [4]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆向きに計算グラフのノードを辿る
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

まずはassert文について説明します。assert文は`assert ...`のように使います。ここで、`...`が`True`でない場合は例外が発生します。そのため、assert文は、条件を満たしているかどうかの確認に利用できます。ちなみに、上のコードを実行すると、問題なく（例外が発生することなく）実行できるので、assert文の条件をすべて満たしていることが分かります。

上のコードが示すように、`Variable`のインスタンス変数`creator`で1つ前の`Function`へ辿ります。そして、`Function`のインスタンス変数`input`で1つ前の`Variable`へ辿ります。このつながりを図示すれば、@<img>{ch1/1-16}のようになります。

<br>![img](images/1-16.png)
<br>**図7-3** `y`を起点とした計算グラフを逆向きに辿る<br><br>

**図7-3**のとおり、私たちの計算グラフは、関数と変数の間の「つながり」によって構築されます。そして重要なことに、その「つながり」は、実際に計算が行われるときに（順伝播でデータを流すときに）作られます。この特徴こそがDefine-by-Runです。つまりは、データを流すことによって「つながり」が作られるのです。

なお、**図7-3**のような「つながり」によるデータ構造は「リンク付きノード」と呼ばれます。ノードはグラフを構成する要素であり、リンクは別のノードへの参照を表します。つまり私たちは、「リンク付きノード」というデータ構造で計算グラフを表現しているのです。

## 7.3 逆伝播を試す

それでは、変数と関数のつながりを利用して、逆伝播を行ってみましょう。まずは`y`から`b`までの逆伝播を行います。これは次のように実装できます（参考のために図も合わせて示します）。

<br>![img](images/1-161.png)

In [5]:
y.grad = np.array(1.0)

C = y.creator  # 1. 関数を取得
b = C.input  # 2. 関数の入力を取得
b.grad = C.backward(y.grad)  # 3. 関数のbackwardメソッドを呼ぶ

ここでは、`y`のインスタンス変数`creator`から関数を取得し、その関数の`input`によって入力変数を取得します。そして、関数の`backward`メソッドを呼び出します。これに続いて、次は変数`b`から`a`への逆伝播の図とコードを示します。

<br>![img](images/1-162.png)

In [6]:
B = b.creator  # 1. 関数を取得
a = B.input  # 2. 関数の入力を取得
a.grad = B.backward(b.grad)  # 3. 関数のbackwardメソッドを呼ぶ

ここでも前と同じロジックで逆伝播を行います。具体的には、

 1. 関数を取得
 2. 関数の入力を取得
 3. 関数の`backward`メソッドを呼ぶ


という流れです。それでは最後に、変数`a`から`x`への逆伝播です。

<br>![img](images/1-163.png)

In [7]:
A = a.creator  # 1. 関数を取得
x = A.input  # 2. 関数の入力を取得
x.grad = A.backward(a.grad)  # 3. 関数のbackwardメソッドを呼ぶ
print(x.grad)

3.297442541400256


以上で、すべての逆伝播が終わりました。

## 7.4 backwardメソッドの追加

すでにお察しのとおり、先ほど示した逆伝播のコードは、同じ処理フローが繰り返し現れます。より正確に言えば、変数から1つ前の変数へと遡る逆伝播のロジックが、すべて同じです。そこで、その繰り返しの処理を自動化できるように、`Variable`クラスに`backward`という新しいメソッドを追加します。

In [8]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. 関数を取得
        if f is not None:
            x = f.input  # 2. 関数の入力を取得
            x.grad = f.backward(self.grad)  # 3. 関数のbackwardメソッドを呼ぶ
            x.backward()  # 自分より1つ前の変数のbackwardメソッドを呼ぶ（再帰）

`backward`メソッドは、これまで繰り返し現れた処理フローとほとんど同じです。具体的には、`Variable`の`creator`から関数を取得し、その関数の入力変数を取り出します。そして、関数の`backward`メソッドを呼び出します。最後に、自分よりも1つ前の変数に対して、その変数の`backward`メソッドを呼び出します。これによって、各変数の`backward`メソッドが再帰的に呼ばれるようになります。

<br><div class="alert alert-info">
<b>【ノート】</b>`Variable`インスタンスの`creator`が`None`の場合、逆伝播はそこでストップします。その場合、`Variable`インスタンスは関数以外の方法で生成されたこと――主にユーザによって与えられた変数であること――を意味します。
</div><br>

それでは、この新しい`Variable`を使って、自動化されたバックプロパゲーションを行ってみましょう。

In [9]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆伝播
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


上記のように、変数`y`の`backward`メソッドを呼べば、自動で逆伝播が行われます。実行して得られる結果も、これまでと同じです。おめでとうございます！ これでDeZeroで最も重要な自動微分のベースは完成しました。